# Task 4.5.1: Advanced Gradio - Building Production-Ready Demos

**Module:** 4.5 - Demo Building & Prototyping  
**Time:** 2-3 hours  
**Difficulty:** ⭐⭐⭐☆☆

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- [ ] Master the Gradio Blocks API for complex, custom layouts
- [ ] Implement session and global state management
- [ ] Create custom themes and styling for professional demos
- [ ] Add authentication to protect your demos
- [ ] Handle events and create interactive component relationships

---

## 📚 Prerequisites

- Completed: Module 3.6 (AI Agents) or familiarity with LLM APIs
- Knowledge of: Python, basic HTML/CSS concepts
- Basic Gradio experience (we'll cover advanced features)

---

## 🌍 Real-World Context

You've built an amazing RAG system that can answer questions about your company's documentation. Your manager says: *"Great work! Can you demo this at the all-hands meeting next week?"*

Suddenly, you need:
- A polished interface that non-technical stakeholders can use
- Multiple tabs for different features (chat, document upload, settings)
- User authentication so only authorized people can access it
- Professional styling that matches company branding

This is where Gradio's advanced features shine. Let's build demos that impress!

---

## 🧒 ELI5: What is Gradio?

> **Imagine you're building a lemonade stand.** You've got the best lemonade recipe (your AI model), but you need a nice table, a sign, cups, and a way to take orders (your interface).
>
> Gradio is like a "lemonade stand kit" for AI. Instead of spending weeks building a website, you snap together pre-made pieces: a text box for questions, a button to submit, a display for answers. In 5 lines of code, you've got a working stand!
>
> **The Blocks API** is like upgrading from a basic stand to a full food truck. Now you can arrange multiple windows, have different stations (tabs), store customer preferences (state), and even require a membership card (authentication).
>
> **In AI terms:** Gradio Blocks lets you create complex, multi-component web interfaces for your ML models without learning web development.

---

## Part 1: Gradio Basics Refresher

Let's start with a quick refresher on basic Gradio, then level up to Blocks.

### The Simple Way: `gr.Interface`

This is Gradio's "easy mode" - perfect for quick demos with one input and one output.

In [ ]:
# First, let's install/upgrade Gradio
# Note: On DGX Spark, use the NGC container for best compatibility
!pip install -q gradio>=4.0.0

In [ ]:
import gradio as gr

# Simple function to demo
def greet(name: str, intensity: int) -> str:
    """Generate an enthusiastic greeting."""
    return f"Hello, {name}!" + "!" * intensity

# The simple way - gr.Interface
simple_demo = gr.Interface(
    fn=greet,
    inputs=[
        gr.Textbox(label="Your Name", placeholder="Enter your name..."),
        gr.Slider(0, 10, value=2, step=1, label="Excitement Level")
    ],
    outputs=gr.Textbox(label="Greeting"),
    title="Simple Greeter",
    description="A basic Gradio demo to show the Interface API."
)

# Launch in notebook (inline=True shows it in the notebook)
# In production, use demo.launch(server_name="0.0.0.0", server_port=7860)
simple_demo.launch(inline=True, share=False)

### 🔍 What Just Happened?

With just a few lines, we created:
- A text input for the name
- A slider for excitement level
- A submit button (automatic)
- An output text box

**But what if we need:**
- Multiple tabs?
- Complex layouts (columns, rows)?
- Components that interact with each other?
- Session state that persists across interactions?

That's where **Blocks** comes in! 🚀

In [ ]:
# Close the previous demo
simple_demo.close()

---

## Part 2: The Blocks API - Full Control

### 🧒 ELI5: Blocks vs Interface

> **Interface** is like ordering a Happy Meal - you get a standard package.
>
> **Blocks** is like a buffet - you pick exactly what you want and arrange it however you like.

### Basic Blocks Structure

In [ ]:
import gradio as gr

# The Blocks context manager
with gr.Blocks() as blocks_demo:
    # Add a title using Markdown
    gr.Markdown("# 🤖 My First Blocks App")
    gr.Markdown("This demonstrates the basic Blocks structure.")
    
    # Components are added in order (top to bottom)
    name_input = gr.Textbox(label="Your Name")
    greet_btn = gr.Button("Say Hello", variant="primary")
    output = gr.Textbox(label="Output")
    
    # Define what happens when button is clicked
    def say_hello(name):
        return f"Hello, {name}! Welcome to Gradio Blocks!"
    
    # Connect the button click to the function
    greet_btn.click(
        fn=say_hello,           # Function to call
        inputs=[name_input],    # Input components
        outputs=[output]        # Output components
    )

blocks_demo.launch(inline=True)

### 🔍 Key Differences from Interface:

1. **Components are variables** - You save each component to a variable
2. **Explicit event binding** - You manually connect events (`.click()`) to functions
3. **Layout control** - Components appear in the order you define them
4. **Multiple buttons** - You can have multiple triggers for different actions

In [ ]:
blocks_demo.close()

---

## Part 3: Complex Layouts with Rows and Columns

Real demos need proper layout. Let's build a more complex interface.

### 🧒 ELI5: Rows and Columns

> Think of your demo like a newspaper. Newspapers use **columns** to fit more text side-by-side, and **rows** to separate different stories.
>
> In Gradio:
> - `gr.Row()` puts things **side by side** (horizontally)
> - `gr.Column()` stacks things **on top of each other** (vertically)
> - You can nest them for complex layouts!

In [ ]:
import gradio as gr

with gr.Blocks() as layout_demo:
    gr.Markdown("# 📐 Layout Demo")
    
    # Row with two equal columns
    with gr.Row():
        with gr.Column():
            gr.Markdown("### Left Panel")
            left_input = gr.Textbox(label="Input A", lines=3)
            left_btn = gr.Button("Process A")
            
        with gr.Column():
            gr.Markdown("### Right Panel")
            right_input = gr.Textbox(label="Input B", lines=3)
            right_btn = gr.Button("Process B")
    
    # Row with unequal columns (scale parameter)
    gr.Markdown("---")
    with gr.Row():
        with gr.Column(scale=2):  # Takes 2/3 of the width
            main_output = gr.Textbox(label="Main Output", lines=5)
            
        with gr.Column(scale=1):  # Takes 1/3 of the width
            sidebar = gr.Textbox(label="Sidebar Info", lines=5)
    
    # Event handlers
    def process_a(text):
        return f"Processed A: {text.upper()}", "From left panel"
    
    def process_b(text):
        return f"Processed B: {text.lower()}", "From right panel"
    
    left_btn.click(process_a, [left_input], [main_output, sidebar])
    right_btn.click(process_b, [right_input], [main_output, sidebar])

layout_demo.launch(inline=True)

In [ ]:
layout_demo.close()

### ✋ Try It Yourself: Three-Column Layout

Create a layout with:
1. A narrow left sidebar (scale=1)
2. A wide center area (scale=3)
3. A narrow right sidebar (scale=1)

<details>
<summary>💡 Hint</summary>

```python
with gr.Row():
    with gr.Column(scale=1):
        # Left sidebar
    with gr.Column(scale=3):
        # Center (main content)
    with gr.Column(scale=1):
        # Right sidebar
```
</details>

In [ ]:
# Your solution here:
# Create a three-column layout with navigation on the left, 
# main content in the center, and info panel on the right

with gr.Blocks() as three_col_demo:
    gr.Markdown("# Three Column Layout")
    
    # TODO: Add your three-column layout here
    pass

# three_col_demo.launch(inline=True)

---

## Part 4: Tabs for Organized Interfaces

When your demo has multiple features, tabs keep things organized.

### 🧒 ELI5: Tabs

> Tabs are like folders in a filing cabinet. Everything is there, but organized so you only see what you need.

In [ ]:
import gradio as gr

with gr.Blocks() as tabs_demo:
    gr.Markdown("# 🗂️ Multi-Tab Demo")
    
    with gr.Tabs():
        # Tab 1: Chat
        with gr.TabItem("💬 Chat"):
            gr.Markdown("### Chat Interface")
            chatbot = gr.Chatbot(height=300)
            msg = gr.Textbox(label="Message", placeholder="Type your message...")
            with gr.Row():
                send_btn = gr.Button("Send", variant="primary")
                clear_btn = gr.Button("Clear")
        
        # Tab 2: Settings
        with gr.TabItem("⚙️ Settings"):
            gr.Markdown("### Model Settings")
            model = gr.Dropdown(
                choices=["llama3.1:8b", "llama3.1:70b", "mistral:7b"],
                value="llama3.1:8b",
                label="Select Model"
            )
            temperature = gr.Slider(0, 1, value=0.7, step=0.1, label="Temperature")
            max_tokens = gr.Slider(100, 2000, value=500, step=100, label="Max Tokens")
            save_btn = gr.Button("Save Settings", variant="primary")
        
        # Tab 3: Documents
        with gr.TabItem("📁 Documents"):
            gr.Markdown("### Upload Documents")
            files = gr.File(
                label="Upload Files",
                file_count="multiple",
                file_types=[".pdf", ".txt", ".md"]
            )
            index_btn = gr.Button("Index Documents", variant="primary")
            status = gr.Textbox(label="Status", interactive=False)
    
    # Event handlers
    def respond(message, history):
        """Simple echo for demo purposes."""
        response = f"Echo: {message}"
        history.append((message, response))
        return history, ""
    
    def clear_chat():
        return [], ""
    
    def save_settings(model, temp, max_tok):
        return f"Settings saved: {model}, temp={temp}, max_tokens={max_tok}"
    
    def index_docs(files):
        if files:
            return f"Indexed {len(files)} files!"
        return "No files uploaded."
    
    send_btn.click(respond, [msg, chatbot], [chatbot, msg])
    msg.submit(respond, [msg, chatbot], [chatbot, msg])  # Also trigger on Enter
    clear_btn.click(clear_chat, None, [chatbot, msg])
    index_btn.click(index_docs, [files], [status])

tabs_demo.launch(inline=True)

In [ ]:
tabs_demo.close()

---

## Part 5: State Management

State is how your demo "remembers" things between interactions.

### 🧒 ELI5: State

> Imagine you're playing a video game. The game needs to remember your score, your inventory, and where you are - even when you pause. That's **state**.
>
> In Gradio:
> - **Session State** (`gr.State`) - Remembers things for ONE user's browser session
> - **Global State** - Remembers things for ALL users (use carefully!)

### Session State Example

In [ ]:
import gradio as gr

with gr.Blocks() as state_demo:
    gr.Markdown("# 🎮 State Demo: Click Counter")
    
    # Session state - each user gets their own counter
    click_count = gr.State(value=0)
    
    with gr.Row():
        counter_display = gr.Number(label="Click Count", value=0, interactive=False)
        
    with gr.Row():
        increment_btn = gr.Button("Click Me! (+1)", variant="primary")
        reset_btn = gr.Button("Reset")
    
    # History of clicks
    history = gr.State(value=[])
    history_display = gr.Textbox(label="Click History", lines=5, interactive=False)
    
    def increment(count, hist):
        """Increment counter and add to history."""
        from datetime import datetime
        new_count = count + 1
        hist.append(f"Click #{new_count} at {datetime.now().strftime('%H:%M:%S')}")
        history_text = "\n".join(hist[-10:])  # Show last 10 entries
        return new_count, new_count, hist, history_text
    
    def reset():
        """Reset counter and history."""
        return 0, 0, [], "History cleared."
    
    increment_btn.click(
        increment,
        inputs=[click_count, history],
        outputs=[click_count, counter_display, history, history_display]
    )
    
    reset_btn.click(
        reset,
        inputs=None,
        outputs=[click_count, counter_display, history, history_display]
    )

state_demo.launch(inline=True)

### 🔍 How State Works:

1. `gr.State(value=0)` creates a hidden component that stores data
2. State is passed as input to functions and returned as output
3. The state value persists across button clicks
4. Each browser session gets its own state (opening in a new tab = new state)

In [ ]:
state_demo.close()

### Practical State: Settings Persistence

A common use case is remembering user settings across tabs:

In [ ]:
import gradio as gr

with gr.Blocks() as settings_demo:
    gr.Markdown("# ⚙️ Settings Persistence Demo")
    
    # Store settings in state
    user_settings = gr.State(value={
        "model": "llama3.1:8b",
        "temperature": 0.7,
        "theme": "light"
    })
    
    with gr.Tabs():
        with gr.TabItem("💬 Chat"):
            gr.Markdown("### Chat (Uses Saved Settings)")
            current_settings = gr.Textbox(
                label="Current Settings", 
                value="Settings: llama3.1:8b, temp=0.7, theme=light",
                interactive=False
            )
            chatbot = gr.Chatbot(height=200)
            msg = gr.Textbox(label="Message")
            send_btn = gr.Button("Send", variant="primary")
            
        with gr.TabItem("⚙️ Settings"):
            gr.Markdown("### Configure Your Preferences")
            model_input = gr.Dropdown(
                choices=["llama3.1:8b", "llama3.1:70b", "mistral:7b"],
                value="llama3.1:8b",
                label="Model"
            )
            temp_input = gr.Slider(0, 1, value=0.7, step=0.1, label="Temperature")
            theme_input = gr.Radio(
                choices=["light", "dark"],
                value="light",
                label="Theme"
            )
            save_btn = gr.Button("Save Settings", variant="primary")
            save_status = gr.Textbox(label="Status", interactive=False)
    
    def save_settings(settings, model, temp, theme):
        """Save settings to state."""
        settings = {
            "model": model,
            "temperature": temp,
            "theme": theme
        }
        settings_text = f"Settings: {model}, temp={temp}, theme={theme}"
        return settings, settings_text, f"✅ Settings saved!"
    
    def chat_respond(message, history, settings):
        """Respond using current settings."""
        response = f"[Using {settings['model']} @ temp={settings['temperature']}] Echo: {message}"
        history.append((message, response))
        return history, ""
    
    save_btn.click(
        save_settings,
        inputs=[user_settings, model_input, temp_input, theme_input],
        outputs=[user_settings, current_settings, save_status]
    )
    
    send_btn.click(
        chat_respond,
        inputs=[msg, chatbot, user_settings],
        outputs=[chatbot, msg]
    )

settings_demo.launch(inline=True)

In [ ]:
settings_demo.close()

---

## Part 6: Custom Themes and Styling

Make your demos look professional with custom themes and CSS.

### 🧒 ELI5: Themes

> A theme is like picking out an outfit for your demo. The built-in themes are like store-bought clothes. Custom themes are like getting something tailored just for you.

### Using Built-in Themes

In [ ]:
import gradio as gr

# List available themes
print("Available Gradio themes:")
print("- gr.themes.Base()")
print("- gr.themes.Default()")
print("- gr.themes.Soft()")
print("- gr.themes.Monochrome()")
print("- gr.themes.Glass()")

In [ ]:
import gradio as gr

# Using the Soft theme
with gr.Blocks(theme=gr.themes.Soft()) as theme_demo:
    gr.Markdown("# 🎨 Theme Demo: Soft Theme")
    gr.Markdown("This uses the built-in Soft theme for a friendly, rounded look.")
    
    with gr.Row():
        with gr.Column():
            name = gr.Textbox(label="Name", placeholder="Enter your name")
            email = gr.Textbox(label="Email", placeholder="email@example.com")
            submit = gr.Button("Submit", variant="primary")
        
        with gr.Column():
            output = gr.Textbox(label="Output", lines=5)
    
    submit.click(
        lambda n, e: f"Name: {n}\nEmail: {e}",
        inputs=[name, email],
        outputs=output
    )

theme_demo.launch(inline=True)

In [ ]:
theme_demo.close()

### Creating Custom Themes

In [ ]:
import gradio as gr

# Create a custom theme
nvidia_theme = gr.themes.Soft(
    primary_hue="green",           # NVIDIA green
    secondary_hue="gray",
    neutral_hue="slate",
    font=gr.themes.GoogleFont("Inter"),
).set(
    # Customize specific properties
    button_primary_background_fill="#76b900",  # NVIDIA green
    button_primary_background_fill_hover="#5a9000",
    button_primary_text_color="white",
    block_title_text_weight="600",
    block_label_text_size="sm",
)

with gr.Blocks(theme=nvidia_theme) as custom_theme_demo:
    gr.Markdown(
        """
        # 💚 Custom NVIDIA-Style Theme
        
        This demo uses a custom theme with NVIDIA green primary color.
        """
    )
    
    with gr.Row():
        with gr.Column():
            model = gr.Dropdown(
                choices=["Llama 3.1 8B", "Mistral 7B", "Nemotron 70B"],
                value="Llama 3.1 8B",
                label="Select Model"
            )
            prompt = gr.Textbox(label="Prompt", lines=3, placeholder="Enter your prompt...")
            generate_btn = gr.Button("Generate", variant="primary")
            
        with gr.Column():
            output = gr.Textbox(label="Response", lines=8)
    
    generate_btn.click(
        lambda m, p: f"[{m}] Generated response for: {p}",
        inputs=[model, prompt],
        outputs=output
    )

custom_theme_demo.launch(inline=True)

In [ ]:
custom_theme_demo.close()

### Custom CSS for Fine-Grained Control

In [ ]:
import gradio as gr

# Custom CSS
custom_css = """
.gradio-container {
    max-width: 1000px !important;
    margin: auto !important;
}

.highlight-box {
    background-color: #fffde7;
    border-left: 4px solid #ffc107;
    padding: 1rem;
    margin: 1rem 0;
    border-radius: 0 8px 8px 0;
}

.success-message {
    background-color: #e8f5e9;
    border-left: 4px solid #4caf50;
    padding: 0.75rem;
    border-radius: 0 8px 8px 0;
}

.main-title {
    text-align: center;
    background: linear-gradient(90deg, #667eea 0%, #764ba2 100%);
    -webkit-background-clip: text;
    -webkit-text-fill-color: transparent;
    font-size: 2.5rem !important;
    font-weight: 700;
}
"""

with gr.Blocks(css=custom_css) as css_demo:
    gr.Markdown("<h1 class='main-title'>Custom CSS Demo</h1>")
    
    gr.HTML("""
        <div class='highlight-box'>
            <strong>💡 Pro Tip:</strong> Custom CSS lets you match any brand or design system!
        </div>
    """)
    
    with gr.Row():
        name = gr.Textbox(label="Your Name")
        greet_btn = gr.Button("Greet Me", variant="primary")
    
    output = gr.HTML(label="Greeting")
    
    def greet(name):
        return f"<div class='success-message'>✅ Hello, <strong>{name}</strong>! Welcome to the custom CSS demo.</div>"
    
    greet_btn.click(greet, inputs=[name], outputs=[output])

css_demo.launch(inline=True)

In [ ]:
css_demo.close()

---

## Part 7: Authentication

Protect your demos from unauthorized access.

### 🧒 ELI5: Authentication

> Authentication is like a bouncer at a club. Before you can enter, you need to show your ID (username and password). No ID = no entry.

### Simple Username/Password Auth

In [ ]:
import gradio as gr

# Simple authentication
def authenticate(username: str, password: str) -> bool:
    """Check if username and password are valid."""
    # In production, use proper password hashing!
    valid_users = {
        "admin": "admin123",
        "demo": "demo123",
        "professor": "spark2024"
    }
    return valid_users.get(username) == password

with gr.Blocks() as auth_demo:
    gr.Markdown("# 🔐 Protected Demo")
    gr.Markdown("This demo requires authentication to access.")
    
    chatbot = gr.Chatbot(height=300)
    msg = gr.Textbox(label="Message")
    send = gr.Button("Send", variant="primary")
    
    def respond(message, history):
        history.append((message, f"Secure response: {message}"))
        return history, ""
    
    send.click(respond, [msg, chatbot], [chatbot, msg])

# Launch with authentication
# Note: In a notebook, auth creates a login page
print("Demo with auth would launch with:")
print('auth_demo.launch(auth=authenticate, auth_message="Welcome! Please login.")')
print("\nFor the notebook demo, we'll skip auth:")
auth_demo.launch(inline=True)

In [ ]:
auth_demo.close()

### Access Control with User Info

In [ ]:
import gradio as gr

# Role-based access control
USER_ROLES = {
    "admin": {"role": "admin", "can_edit": True, "can_delete": True},
    "editor": {"role": "editor", "can_edit": True, "can_delete": False},
    "viewer": {"role": "viewer", "can_edit": False, "can_delete": False},
}

def get_user_info(request: gr.Request):
    """
    Get user information from the request.
    In production, this would come from auth.
    """
    # This would normally come from authentication
    # For demo, we simulate different users
    return USER_ROLES.get("admin", USER_ROLES["viewer"])

with gr.Blocks() as rbac_demo:
    gr.Markdown("# 👥 Role-Based Access Demo")
    
    user_info = gr.State(value=USER_ROLES["viewer"])
    
    # Role selector (for demo - in production this comes from auth)
    role_selector = gr.Dropdown(
        choices=["admin", "editor", "viewer"],
        value="viewer",
        label="Select Role (Demo Only)"
    )
    
    role_display = gr.Markdown("**Current Role:** Viewer")
    
    with gr.Row():
        content = gr.Textbox(label="Content", lines=3, value="Sample content...")
        
    with gr.Row():
        edit_btn = gr.Button("Edit", variant="secondary")
        delete_btn = gr.Button("Delete", variant="stop")
    
    status = gr.Textbox(label="Status", interactive=False)
    
    def change_role(role):
        info = USER_ROLES[role]
        return info, f"**Current Role:** {role.title()} (Edit: {info['can_edit']}, Delete: {info['can_delete']})"
    
    def try_edit(content, info):
        if info["can_edit"]:
            return f"✅ Content edited: {content[:50]}..."
        return "❌ Permission denied. You need Editor or Admin role to edit."
    
    def try_delete(info):
        if info["can_delete"]:
            return "✅ Content deleted!"
        return "❌ Permission denied. You need Admin role to delete."
    
    role_selector.change(change_role, [role_selector], [user_info, role_display])
    edit_btn.click(try_edit, [content, user_info], [status])
    delete_btn.click(try_delete, [user_info], [status])

rbac_demo.launch(inline=True)

In [ ]:
rbac_demo.close()

---

## Part 8: Advanced Event Handling

### Component Interactions

Components can interact with each other in complex ways.

In [ ]:
import gradio as gr

with gr.Blocks() as events_demo:
    gr.Markdown("# ⚡ Advanced Events Demo")
    
    with gr.Row():
        with gr.Column():
            # Live preview as you type
            text_input = gr.Textbox(
                label="Type something",
                placeholder="Watch the preview update as you type..."
            )
            char_count = gr.Number(label="Character Count", value=0, interactive=False)
            
        with gr.Column():
            preview = gr.Markdown("**Preview:** (start typing)")
    
    gr.Markdown("---")
    
    with gr.Row():
        # Chained dropdowns
        category = gr.Dropdown(
            choices=["Animals", "Colors", "Countries"],
            label="Category",
            value="Animals"
        )
        item = gr.Dropdown(
            choices=["Dog", "Cat", "Bird"],
            label="Item",
            value="Dog"
        )
    
    selection = gr.Textbox(label="Selection", interactive=False)
    
    # Event: Update preview and count as user types
    def update_preview(text):
        if not text:
            return "**Preview:** (start typing)", 0
        return f"**Preview:** {text}", len(text)
    
    text_input.change(
        update_preview,
        inputs=[text_input],
        outputs=[preview, char_count]
    )
    
    # Event: Update second dropdown based on first
    def update_items(category):
        options = {
            "Animals": ["Dog", "Cat", "Bird", "Fish"],
            "Colors": ["Red", "Green", "Blue", "Yellow"],
            "Countries": ["USA", "Canada", "UK", "Japan"]
        }
        choices = options.get(category, [])
        return gr.Dropdown(choices=choices, value=choices[0] if choices else None)
    
    category.change(
        update_items,
        inputs=[category],
        outputs=[item]
    )
    
    # Event: Update selection when item changes
    def show_selection(cat, itm):
        return f"You selected: {itm} from {cat}"
    
    item.change(
        show_selection,
        inputs=[category, item],
        outputs=[selection]
    )

events_demo.launch(inline=True)

In [ ]:
events_demo.close()

### Progress Indicators and Streaming

In [ ]:
import gradio as gr
import time

with gr.Blocks() as streaming_demo:
    gr.Markdown("# 🌊 Streaming Response Demo")
    
    prompt = gr.Textbox(label="Prompt", placeholder="Enter a topic...")
    
    with gr.Row():
        stream_btn = gr.Button("Generate (Streaming)", variant="primary")
        regular_btn = gr.Button("Generate (Wait for Full)")
    
    output = gr.Textbox(label="Output", lines=10)
    
    def generate_streaming(prompt):
        """
        Generator function for streaming output.
        Yields chunks of text as they become available.
        """
        response = f"Writing about: {prompt}\n\n"
        yield response
        
        sentences = [
            "This is the first sentence of our response. ",
            "Here's some more content being generated. ",
            "Notice how the text appears chunk by chunk. ",
            "This simulates a streaming LLM response. ",
            "The user sees progress instead of waiting. ",
            "This improves perceived responsiveness! "
        ]
        
        for sentence in sentences:
            time.sleep(0.5)  # Simulate generation time
            response += sentence
            yield response
        
        yield response + "\n\n✅ Generation complete!"
    
    def generate_regular(prompt):
        """
        Regular function that waits until complete.
        """
        time.sleep(3)  # Simulate full generation time
        return f"Writing about: {prompt}\n\n" + \
               "This is the complete response that appears all at once. " + \
               "The user had to wait for the entire generation to complete. " + \
               "This can feel slow for longer responses.\n\n" + \
               "✅ Generation complete!"
    
    # Note: streaming is automatic when function is a generator
    stream_btn.click(
        generate_streaming,
        inputs=[prompt],
        outputs=[output]
    )
    
    regular_btn.click(
        generate_regular,
        inputs=[prompt],
        outputs=[output]
    )

streaming_demo.launch(inline=True)

In [ ]:
streaming_demo.close()

---

## ⚠️ Common Mistakes

### Mistake 1: Forgetting to Return State

```python
# ❌ Wrong - state is lost
def bad_handler(count):
    count += 1
    return count  # Missing state update!

# ✅ Right - return state as output
def good_handler(count):
    count += 1
    return count, count  # Return to state AND display
```

### Mistake 2: Blocking the Event Loop

```python
# ❌ Wrong - blocks the UI
def slow_function(text):
    time.sleep(30)  # UI freezes!
    return result

# ✅ Right - use a generator for long operations
def streaming_function(text):
    for chunk in generate_chunks(text):
        yield chunk  # UI stays responsive
```

### Mistake 3: Wrong Input/Output Count

```python
# ❌ Wrong - function returns 2, but only 1 output specified
def bad_function(x):
    return x, x*2

btn.click(bad_function, [input], [output])  # Error!

# ✅ Right - match returns to outputs
btn.click(bad_function, [input], [output1, output2])
```

### Mistake 4: Not Handling Empty Inputs

```python
# ❌ Wrong - crashes on empty input
def bad_process(text):
    return text.upper()  # Error if text is None!

# ✅ Right - handle edge cases
def good_process(text):
    if not text:
        return "Please enter some text."
    return text.upper()
```

---

## 🎉 Checkpoint

You've learned:
- ✅ The Blocks API for complex, custom layouts
- ✅ Rows, Columns, and Tabs for organizing components
- ✅ Session state for persistent data
- ✅ Custom themes and CSS for professional styling
- ✅ Authentication for protected demos
- ✅ Advanced event handling and streaming

---

## 🚀 Challenge (Optional)

Build a **Document Q&A Demo** with:
1. Three tabs: Upload, Chat, Settings
2. Custom NVIDIA-green theme
3. Session state to remember uploaded files
4. Streaming responses in the chat
5. User authentication

This is similar to Lab 4.5.1, but try to build it from scratch!

---

## 📖 Further Reading

- [Gradio Blocks Documentation](https://gradio.app/docs/blocks)
- [Gradio Theming Guide](https://gradio.app/guides/theming-guide)
- [Gradio State Management](https://gradio.app/guides/state-in-blocks)
- [Gradio Authentication](https://gradio.app/guides/sharing-your-app#authentication)

---

## 🧹 Cleanup

In [ ]:
# Clean up any running demos
import gc

# Force garbage collection
gc.collect()

print("✅ Cleanup complete!")